# Structured Tool Chat Agent

This notebook walks through using a chat agent capable of using multi-input tools.

Older agents are configured to specify an action input as a single string, but this agent can use the provided tools' `args_schema` to populate the action input.

This functionality is natively available in the (`structured-chat-zero-shot-react-description` or `AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`).

In [1]:
import os
os.environ["LANGCHAIN_TRACING"] = "true" # If you want to trace the execution of the program, set to "true"

In [1]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

### Initialize Tools

We will test the agent using a web browser.

In [2]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser, # A synchronous browser is available, though it isn't compatible with jupyter.
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [3]:
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()

In [4]:
llm = ChatOpenAI(temperature=0) # Also works well with Anthropic models
agent_chain = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [5]:
response = await agent_chain.arun(input="Hi I'm Erica.")
print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Hello Erica, how can I assist you today?"
}
```


> Finished chain.
Hello Erica, how can I assist you today?


In [7]:
response = await agent_chain.arun(input="Don't need help really just chatting.")
print(response)



> Entering new AgentExecutor chain...

> Finished chain.
I'm here to chat! How's your day going?


In [8]:
response = await agent_chain.arun(input="Browse to blog.langchain.dev and summarize the text, please.")
print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev/"
  }
}
```


Observation: Navigating to https://blog.langchain.dev/ returned status code 200
Thought:I need to extract the text from the webpage to summarize it.
Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```

Observation: LangChain LangChain Home About GitHub Docs LangChain The official LangChain blog. Auto-Evaluator Opportunities Editor's Note: this is a guest blog post by Lance Martin.


TL;DR

We recently open-sourced an auto-evaluator tool for grading LLM question-answer chains. We are now releasing an open source, free to use hosted app and API to expand usability. Below we discuss a few opportunities to further improve May 1, 2023 5 min read Callbacks Improvements TL;DR: We're announcing improvements to our callbacks system, which powers logging, tracing, streaming output, and some awesome third-party integrations

In [9]:
response = await agent_chain.arun(input="What's the latest xkcd comic about?")
print(response)



> Entering new AgentExecutor chain...
Thought: I can navigate to the xkcd website and extract the latest comic title and alt text to answer the question.
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://xkcd.com/"
  }
}
```

Observation: Navigating to https://xkcd.com/ returned status code 200
Thought:I can extract the latest comic title and alt text using CSS selectors.
Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "#ctitle, #comic img",
    "attributes": ["alt", "src"]
  }
}
``` 

Observation: [{"alt": "Tapetum Lucidum", "src": "//imgs.xkcd.com/comics/tapetum_lucidum.png"}]
Thought:
> Finished chain.
The latest xkcd comic is titled "Tapetum Lucidum" and the image can be found at https://xkcd.com/2565/.


## Adding in memory

Here is how you add in memory to this agent

In [6]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [7]:
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [8]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    memory=memory, 
    agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"]
    }
)

In [9]:
response = await agent_chain.arun(input="Hi I'm Erica.")
print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Hi Erica! How can I assist you today?"
}
```


> Finished chain.
Hi Erica! How can I assist you today?


In [10]:
response = await agent_chain.arun(input="whats my name?")
print(response)



> Entering new AgentExecutor chain...
Your name is Erica.

> Finished chain.
Your name is Erica.
